# Testing the Challenge

This notebook is used to apply the model to the challenge **with the mini-models**. This is using preformatted data that includes the features used for model training.

In [1]:
import numpy as np
import pandas as pd
import lightgbm as lgbm

from matplotlib import pyplot


In [2]:
'''
Load the data
'''
data = pd.read_sas('data/who_rated_what_ans_wvars.sas7bdat')
answer_data = pd.read_csv('answer_data.csv')

In [3]:
'''
Set variable data to to match the correct rating value
'''
data.loc[(answer_data["Rating"] == 0),"Rated"] = 0

data.head()

,User_ID,Movie_ID,Year,Rated,Ratings_for_Movie,Ratings_for_Movie_2005,Netflix_Release_Year,Movie_Rating_Time,Movie_Ratings_per_Day,Release_Year,AVG_Rating_for_Movie,Ratings_from_User,Ratings_from_User_2005,AVG_Rating_from_User,User_Rating_Time,User_Ratings_per_Day,User_Entry_Year
0,6.0,16983.0,2006.0,0.0,546.0,222.0,2000.0,5.965777,0.250574,1987.0,3.474359,626.0,310.0,3.418530,1.738535,0.985827,2004.0
1,10.0,15957.0,2006.0,0.0,2122.0,1198.0,2003.0,2.872005,2.022879,1998.0,3.905278,260.0,91.0,3.392308,2.847365,0.250000,2002.0
2,10.0,14584.0,2006.0,1.0,37492.0,18285.0,1999.0,6.004107,17.096215,1999.0,3.819455,260.0,91.0,3.392308,2.847365,0.250000,2002.0
3,10.0,11888.0,2006.0,0.0,51611.0,28415.0,1999.0,6.056126,23.332278,1993.0,3.790239,260.0,91.0,3.392308,2.847365,0.250000,2002.0
4,131.0,17405.0,2006.0,0.0,58563.0,58562.0,2004.0,1.002053,160.008197,2005.0,3.523761,131.0,131.0,3.175573,0.717317,0.500000,2005.0


In [4]:
'''
Preprocess the data (separate features from labels)
'''
X = data.drop(['User_ID', 'Movie_ID', 'Rated'], axis=1)
Y = data['Rated'].values

In [5]:
'''
Print available models
'''

import os

print(next(os.walk('models'))[2])

['m50M_500.txt', 'm25M_400.txt', 'm50M_250.txt', 'model.txt', 'm25M_250.txt', 'm1000_002.txt', 'm1000.txt', 'm50M_1K.txt', 'm1.txt']


In [9]:
'''
Load the models
'''
models = []
for m in next(os.walk('models/mini_models'))[2]:
    models.append(lgbm.Booster(model_file = 'models/mini_models/' + m))

In [11]:
'''
Test each model accuracy
'''
from sklearn.metrics import accuracy_score

predictions = []

for model in models:
    p = model.predict(X)
    print("Accuracy: {}".format(accuracy_score(Y, p > 0.5)))
    predictions.append(p)

Accuracy: 0.80016
Accuracy: 0.79732
Accuracy: 0.79569
Accuracy: 0.79941
Accuracy: 0.79515


In [14]:
from sklearn.metrics import classification_report

for p in predictions:
    print(classification_report(Y, p > 0.5))

              precision    recall  f1-score   support

         0.0       0.93      0.85      0.89     92196
         1.0       0.11      0.22      0.15      7804

    accuracy                           0.80    100000
   macro avg       0.52      0.53      0.52    100000
weighted avg       0.86      0.80      0.83    100000

              precision    recall  f1-score   support

         0.0       0.93      0.85      0.89     92196
         1.0       0.11      0.22      0.15      7804

    accuracy                           0.80    100000
   macro avg       0.52      0.53      0.52    100000
weighted avg       0.86      0.80      0.83    100000

              precision    recall  f1-score   support

         0.0       0.93      0.84      0.88     92196
         1.0       0.11      0.22      0.15      7804

    accuracy                           0.80    100000
   macro avg       0.52      0.53      0.51    100000
weighted avg       0.86      0.80      0.83    100000

              preci

In [29]:
# use mean prediction

print(classification_report(Y, np.mean(predictions, axis=0) > 0.5))

              precision    recall  f1-score   support

         0.0       0.93      0.89      0.91     92196
         1.0       0.12      0.19      0.15      7804

    accuracy                           0.83    100000
   macro avg       0.52      0.54      0.53    100000
weighted avg       0.86      0.83      0.85    100000



The mini model does not perform better. The models are too similar to get a meaningful boost from averaging.

In [31]:
# RMSE
print("LightGBM RSME:\t{}".format(np.sqrt(np.mean(np.power(Y-(np.mean(predictions, axis=0) > 0.5), 2)))))
print("All zeros RSME:\t{}".format(np.sqrt(np.mean(np.power(Y-0, 2)))))

LightGBM RSME:	0.41158231254513356
All zeros RSME:	0.2793564031841762


The model underperforms guessing 0's for RMSE. Probable fixes would be to add more features and train more data for longer.

In [14]:
from sklearn.metrics import confusion_matrix

matrix = confusion_matrix(Y, predictions > 0.5)

pd.DataFrame(matrix)

,0,1
0,80274,11922
1,6215,1589


The model too often guesses "1" for ground truth "0".

The missing data is more features. The model performance for adding more training steps and data is almost identical.